In [18]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pickle
import nltk
nltk.download('punkt')
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from string import punctuation
import numpy as np
import re
from sklearn import feature_extraction
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from nltk.stem import PorterStemmer
from nltk.corpus import words

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/austinkrause/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [8]:
#load in our xgb classification model
xgb = pickle.load(open("xgboost_model.sav", "rb"))

In [14]:
def create_soup(url):
    facts = requests.get(url)
    soup = BeautifulSoup(facts.content, 'html.parser')
    return soup

In [209]:
def get_article_text(url):
    #url = input('Enter a url: \n')
    soup = create_soup(url)
    
    text = []
    for i in soup.find_all('p'):
        text.append(i.get_text())

    full_text = ' '.join(text).replace('\xa0', '')
#     word_count_ = full_text.count(' ')
#     gensim_summary = summarize(full_text, word_count = word_count_/10).strip('. ').replace('.\n', '')
    
#     print('\n Full Text:\n', full_text, '\n\n')
#     print('Extractive Summary: \n', gensim_summary)
    return full_text

In [234]:
#get_article_text('https://news.yahoo.com/cambodia-islands-under-threat-woman-trying-save-them-144633896.html')



In [39]:
#set stop words
stops = list(set(stopwords.words('english'))) + list(punctuation)

In [43]:
stemmer = PorterStemmer()

In [99]:
#helper function to remove stop words
def remove_stops(text):
    text_no_stops = []
    for i in text:
        i = i.lower()
        if i not in stops:
            if len(i) == 1:
                pass
            else:
                text_no_stops.append(i)
        else:
            pass
    return text_no_stops

In [35]:
#helper function to lemmatize text
def lemmatize_text(text):
    lemmatized = []
    for word in text:
        lemmatized.append(lemmatizer.lemmatize(word))
    return lemmatized

In [116]:
#create helper function to stem each word in a list and concat the list
def stem_list(lst):
    stemmed_list = []
    for i in lst:
        stemmed_list.append(stemmer.stem(i))
    stem_string = ' '.join(stemmed_list[:100])
    return stem_string

In [117]:
#function to perform lemmatization, stemming, and final preprocessing
def xgb_text_prep(sample_text):
    tokens = word_tokenize(str(sample_text), language = 'en')
    no_stops = remove_stops(tokens)
    stemmed = stem_list(no_stops)
    #stemmed = stemmed.replace('\\n', '')
    return [stemmed]
    #return no_stops

In [118]:
xgb_text_prep(test_1)

['hand market stall line narrow concret path serv island main arteri offer usual assort good found across rural cambodia cheap noodl stack coca-cola bottl fake soccer jersey household clean product plastic bag carri home.. litter floor pile squeez sugarcan empti clam shell ocean plastic wrapper bottl cup straw empti bag swept way sight stark remind challeng resid koh sdach island surround archipelago face live harmoni environment.. five minut boat ride mainland cambodia koh sdach surround clear blue water sustain fish commun gener decad overfish indiscrimin wast dispos taken steep toll environment.. recommend eat half-truth teller aliv mariann williamson take trump']

In [119]:
#function for xgboost prdiction
def get_xgb_prediction(text):
    preprocess = text_prep([text])
    return xgb.predict(preprocess)[0]

In [240]:
#find cosine similarities between sentences
def find_similarities(text):
    #tokenize sentences
    sentences = sent_tokenize(text, language = 'en')
    
    #vectorize sentences
    vectorizer = TfidfVectorizer(stop_words = stops)
    trsfm=vectorizer.fit_transform(sentences)
    #creat df for article
    text_df = pd.DataFrame(trsfm.toarray(),columns=vectorizer.get_feature_names(),index=sentences)
    
    #declare how many sentences to use in summary
    num_sentences = text_df.shape[0]
    num_summary_sentences = int(np.ceil(num_sentences**.5))
        
    #find cosine similarity for all sentences
    similarities = cosine_similarity(trsfm, trsfm)
    #print(similarities)
    #create list to hold avg cosine similarities for each sentence
    avgs = []
    for i in similarities:
        avgs.append(i.mean())
     
    #find index values of the sentences to be used for summary
    top_idx = np.argsort(avgs)[-num_summary_sentences:]
    
    return top_idx

In [241]:
def build_summary(text):
    sents_for_sum = find_similarities(text)
    sort = sorted(sents_for_sum)
    print('\n Sentences selected:', sort)
    
    sent_list = sent_tokenize(text)
    print('\n Total sentences:', len(sent_list))
    
    sents = []
    for i in sort:
        sents.append(sent_list[i].replace('\n', ''))
    
    summary = ' '.join(sents)
    return summary

In [242]:
build_summary(test_1)


 Sentences selected: [1, 6, 10, 11, 22, 24, 43]

 Total sentences: 49


'Swept out of the way, but not out of sight, it’s a stark reminder of the challenges residents of Koh Sdach island and the surrounding archipelago face in living harmoniously with their environment.. Just a five minute boat ride from mainland Cambodia, Koh Sdach is surrounded by clear blue waters that have sustained fishing communities for generations. But the plastic problem in these island communities is not just environmental. That doesn’t dissuade Oeun Sina, the loudest voice calling for better environmental practices on land and sea.. As deputy village chief on the island, Ms. Oeun Sina is trying to usher in a sea change in the way the 3,000 residents of her small fishing village see their island and the waters that surround it. She has taken it upon herself to educate her neighbors about recycling and trash disposal, and joins otherwise all-male community sea patrols in search of illegal fishing.. “I want my community to understand about plastics and not to fish illegally,” she s

In [243]:
def xgb_and_summary():
    url = input('Enter a url: \n')
    text = get_article_text(url)
    print('\n')
    xgb_preprocess = xgb_text_prep(text)
    print('This article was placed into cluster:', xgb.predict(xgb_preprocess)[0])
    
    summary = build_summary(text)
    return summary

TEST ARTICLE 1 --- Supreme Court --- returns cluster 7
https://news.yahoo.com/u-supreme-court-spurns-virginia-143413283.html;_ylt=AwrC0COK7wddIj8AGCPQtDMD;_ylu=X3oDMTEydGhybm9tBGNvbG8DYmYxBHBvcwMxBHZ0aWQDQjc2MDlfMQRzZWMDc3I-

TEST ARTICLE 2 --- Donald Trump --- returns cluster 1
https://news.yahoo.com/trump-says-d-foreigners-offer-233911079.html;_ylt=AwrC1C7Y8AddADIAoS_QtDMD;_ylu=X3oDMTEyaTE5czNyBGNvbG8DYmYxBHBvcwM0BHZ0aWQDQjc2MDlfMQRzZWMDc3I-

TEST ARTICLE 3 --- Syria --- returns cluster 11
https://news.yahoo.com/turkish-outpost-syria-shelled-syrian-080926968.html;_ylt=AwrC0wwt8gdd2WgAkwHQtDMD;_ylu=X3oDMTEydGhybm9tBGNvbG8DYmYxBHBvcwMxBHZ0aWQDQjc2MDlfMQRzZWMDc3I-

TEST ARTICLE 4 --- Birth Control Health Insurance --- returns cluster 3
https://news.yahoo.com/democrat-warren-wants-7-billion-134454649.html;_ylt=AwrC1DGZ8gddwlsAJwLQtDMD;_ylu=X3oDMTEydDBqZzZmBGNvbG8DYmYxBHBvcwMyBHZ0aWQDQjc2MDlfMQRzZWMDc3I-

TEST ARTICLE 5 --- Bernie Sanders --- returns cluster 3
https://news.yahoo.com/sanders-outline-democratic-socialism-means-045302693.html;_ylt=AwrC0CMD4whdTSoALAHQtDMD;_ylu=X3oDMTEydDBqZzZmBGNvbG8DYmYxBHBvcwMyBHZ0aWQDQjc2MDlfMQRzZWMDc3I-

In [244]:
xgb_and_summary()

Enter a url: 
https://news.yahoo.com/democrat-warren-wants-7-billion-134454649.html;_ylt=AwrC1DGZ8gddwlsAJwLQtDMD;_ylu=X3oDMTEydDBqZzZmBGNvbG8DYmYxBHBvcwMyBHZ0aWQDQjc2MDlfMQRzZWMDc3I-


This article was placed into cluster: 3

 Sentences selected: [0, 5, 7, 9]

 Total sentences: 12


'By Ginger Gibson WASHINGTON (Reuters) - Democratic presidential hopeful Elizabeth Warren is proposing creating a $7 billion fund to provide grants to help more minorities start their own business, she announced on Friday. The fund would provide grants - not loans or loan guarantees - to minority entrepreneurs, Warren is proposing. Warren would have the program administered by state and local governments. Additionally, Warren wants to require the states and local governments administering her program to hire more minority investment managers.'